# Sensitivity Analysis

In [ ]:
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt
import pickle

In [ ]:
OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period

In [ ]:
# file = open('delta_filter_ss.pickle', 'rb')
# [A,B,C,D,T0,Ts,f,ts] = pickle.load(file)
# file.close()

$\large \delta = (e^{\frac{-j2\pi\omega}{fs}} - 1) \times \frac{1}{ts}$

$H(\delta)=\tilde{C_{\delta}}(\delta I-\tilde{A_{\delta}})\tilde{B_{\delta}}+D_{\delta}$

In [ ]:
def delta_bode(A,B,C,D,f,ts):
  q     = C.shape[0]
  p     = B.shape[1]
  fs    = 1/ts
  mag   = np.zeros((q,p,f.shape[0]))
  phz   = np.zeros((q,p,f.shape[0]))
  delta = (np.exp(1j*2*np.pi*(f/fs))-1)/ts

  for i in range(f.shape[0]):
      A_d = delta[i] * np.eye(A.shape[0]) - A
        
      [A_d, T_d] = linalg.matrix_balance(A_d)
      B_d = linalg.solve(T_d, B)
      C_d = C @ T_d
      
      h   = linalg.solve(A_d, B_d)
      h   = (C_d @ h) + D
      mag[:,:,i] = np.abs(h)
      phz[:,:,i] = 180*np.arctan2(np.imag(h),np.real(h))/np.pi
  return mag, phz

In [ ]:
def ss_concat_outputs(sys_0, sys_1):
  A = linalg.block_diag(sys_0.A, sys_1.A)
  B = np.vstack((sys_0.B       , sys_1.B))
  C = linalg.block_diag(sys_0.C, sys_1.C)
  D = np.vstack((sys_0.D       , sys_1.D))
  return control.ss(A,B,C,D)

## SD dIIR Sensitivity
def SD_dIIR_sensitivity(A,B,C,D,T0,Ts,f,ts):

In [ ]:
def SD_dIIR_sensitivity(A,B,C,D,T0,Ts,f,ts):
  # %Sensitivity due to feedthrough coefficient
  B_beta = np.zeros((1, A.shape[1])).T
  B_beta[0][0] = 1
  B_beta = A @ T0 @ B_beta
  S_beta0 = control.ss(A,B_beta,C,1)

  # %Sensitivity due to numerator coefficients
  S_beta = control.ss(A.T,C.T,Ts.T @ T0.T,0)

  # %Sensitivity due to denominator coefficients
  H = control.ss(A,B,C,D)
  S_alpha = control.series(H, S_beta)
  S_bsys = ss_concat_outputs(S_beta0, S_beta)
  null_sys = control.ss(0, 0, 0, 0)
  S_asys = ss_concat_outputs(null_sys, S_alpha)

  [S_bmag,S_bphz] = delta_bode(S_bsys.A,S_bsys.B,S_bsys.C,S_bsys.D,f,ts)
  [S_amag,S_aphz] = delta_bode(S_asys.A,S_asys.B,S_asys.C,S_asys.D,f,ts)
  S_mag = np.squeeze(S_bmag + S_amag)
  S_phz = np.squeeze(S_bphz + S_aphz)

  return S_mag, S_phz


In [ ]:
%store -r
[Ad,Bd,Cd,Dd,T0,Ts,f,ts] = the_delta_filter

SD_dIIR_sensitivity(Ad,Bd,Cd,Dd,T0,Ts,f,ts)